In [1]:
# small script to clean pcl data
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import glob
import os
import numpy as np
import seaborn as sns

In [3]:
pclNotClean = pd.read_csv('/home/oad4/RCF_Movies/pclScores.csv', dtype={'subject': object})
pclNotClean.head()

,group,subject,pcl_An,pcl_Av,pcl_total
0,PTSD,035,7.0,6.0,43.0
1,CC,011,0.0,0.0,0.0
2,PTSD,040,5.0,4.0,24.0
3,PTSD,029,0.0,1.0,11.0
4,CC,020,0.0,0.0,0.0


In [21]:
pclNotClean = pclNotClean[pclNotClean.group!='FPTSD']


0

In [25]:
# read all items pcl
pclAll = pd.read_csv('/home/oad4/RCF_Movies/pclAll_clean.csv')
# remove white space from sume subject numbers
pclAll['subject_id'] = pclAll['subject_id'].str.replace(" ","")
pclAll = pclAll[pclAll.subject_id!='rcf1214']

In [4]:
sns.histplot(behav_data.pcl_total, kde=True)

NameError: name 'behav_data' is not defined

In [27]:
# change to sub-subnumber
for i, row in pclNotClean.iterrows():
    sub = row.subject
    pclNotClean.at[i,'subject'] = 'sub-' + str(sub)

for i, row in pclAll.iterrows():
    sub = row.subject_id.split('RCF')[1]
    pclAll.at[i,'subject_id'] = 'sub-' + str(sub)

1359
003
008
009
010
013
014
015
016
020
021
022
025
026
028
029
030
031
033
034
035
037
038
040
043
044
045
046
047
048
049
049(2)
050
1005
1072
1074
1099
1200
1201
1204
1205
1206
1208
1210
1211
1212
1213
1216
1217
1218
1221
1222
1223
1224
1226
1227
1228
1229
1230
1232
1233
1237
1238
1243
1245
1246
1247
1249
1251
1254
1255
1256
1258
1262
1266
1267
1268
1269
1271
1272
1273
1279
1280
1281
1282
1284
1287
1289
1291
1301
1303
1304
1305
1306
1309
1311
1312
1313
1314
1317
1318
1319
1320
1321
1325
1326
1329
1337
1338
1343
1344
1345
1346
1347
1349
1350
1352
1357
1361
1362
1366
1367
1373
1374
1376
1378
1379
1382
1384
1386
1388
1389
1392
1393
1394
1396
1397
1402
1404
1407
1418
1419
1423
1431
1432
1438
1440
1444
1445
1449
1452
1453
1457
1460
1476
1513
1531
1538
1543


In [17]:
behav_data = pclNotClean

In [20]:
# get extracted time-series and remove 1392
data_dir = '/gpfs/gibbs/pi/levy_ifat/Or/RCF_clips/derivatives'
result = glob.glob(os.path.join(data_dir, 'fmriprep', '*', 'ses-3', 'func', f"*_Average_ROI_n200.csv"))

sub_timeseries = {}
for file in result:
    sub = os.path.basename(file).split('_')[0]
    clip = pd.read_csv(file)
    # remove last frames
    clip = clip[0:1010]
    sub_data = clip
    sub_data.reset_index(inplace=True, drop=True)
    sub_timeseries[sub] = sub_data
## need to remove subject 1392
del sub_timeseries['sub-1392']

In [21]:
subj_list = behav_data["subject"]
remove_subs = [] # add vector of subjects that don't have Brain data
data = []
for subj in subj_list:
    if subj in sub_timeseries:
        sub_data = sub_timeseries[subj]
        data.append(sub_data.values)
    else: 
        print(f'Subject {subj} has no data')
        remove_subs.append(subj)
data = np.array(data)

Subject sub-035 has no data
Subject sub-011 has no data
Subject sub-038 has no data
Subject sub-1216 has no data
Subject sub-1218 has no data
Subject sub-1246 has no data
Subject sub-1247 has no data
Subject sub-1280 has no data
Subject sub-1290 has no data
Subject sub-1318 has no data
Subject sub-1319 has no data
Subject sub-1337 has no data
Subject sub-1343 has no data
Subject sub-1346 has no data
Subject sub-1347 has no data
Subject sub-1357 has no data
Subject sub-1373 has no data
Subject sub-1376 has no data
Subject sub-1378 has no data
Subject sub-1384 has no data
Subject sub-1388 has no data
Subject sub-1392 has no data
Subject sub-1393 has no data
Subject sub-1423 has no data
Subject sub-1431 has no data
Subject sub-1432 has no data
Subject sub-1440 has no data
Subject sub-1444 has no data
Subject sub-1445 has no data
Subject sub-1449 has no data
Subject sub-1457 has no data


In [22]:
# clean behav data to include just the ones with brain
behav_data = behav_data[~np.isin(behav_data.subject, remove_subs)]
behav_data.to_csv("cleanPCL.csv", index=False)
#behav = behav_data["caps_totalscorem"]
#behav_rank = rankdata(behav)